In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Read the CSV file
df = pd.read_csv('dataset.csv')

# Shuffle the data
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,Sentence,Type,Factual/Subjective,Sentiment
0,The chemical symbol for gold is Au.,Affirmation,Factual,Neutral
1,Unfiltered opinions offend me.,Affirmation,Subjective,Anger
2,Exercise is good for health.,Affirmation,Factual,Neutral
3,Spring flowers bloom beautifully.,Affirmation,Factual,Neutral
4,Scientific discoveries excite me!,Affirmation,Subjective,Euphoria


In [4]:
# Verify Class balance "Type" column
df['Type'].value_counts()  

Affirmation    1213
Negation        429
Name: Type, dtype: int64

In [5]:
# Verify Class balance "Factual/Subjective" column
df['Factual/Subjective'].value_counts()  

Subjective    1010
Factual        632
Name: Factual/Subjective, dtype: int64

In [6]:
# Verify Class balance "Sentiment" column
df['Sentiment'].value_counts()  

Neutral      642
Anger        292
Happiness    286
Sadness      260
Euphoria     161
Neutral        1
Name: Sentiment, dtype: int64